In [1]:
import rasterio

## Writing an image

We have so far covered how to read in an image using `rasterio`, although this is only one key technique. It is also essential we know how to write out any image data we produce. 

The trick here, when working with our current `numpy` array structure, is getting the data layers in the right order. Let's look at an example. 

To write the data we will be using the `rasterio.open` function. We will need to set some basic parameters:
- A filename for our new file.
- The mode, which for writing is 'w'.
- The driver which is likely to be 'GTiff' for a GeoTIFF .tif image file.
- Image height.
- Image width.
- The count of the number of dataset bands.
- The Coordinate Reference System
- The affine transformation mapping the pixel space to geographic space.
- The datatype. 
- Any pixel value that should be interpreted as not valid data.

Let's try write our `rgb` (red, green and blue) layer. 

First we need to check the shape of the data. We must make sure our arrays are ordered as bands, columns and then rows. For the sake of clarity, let's load in the data again. 


In [19]:
# This is all code we have already covered!!
# We are rerunning to ensure a clean reload

# ------------------------------------------
import numpy as np
import rasterio

def scale(band): # scale values for display purposes
    return band #/ 10000.0

image_filename = "../week3/20190321_174348_0f1a_3B_AnalyticMS.tif"
my_image = rasterio.open(image_filename)

blue = scale(my_image.read(1))
green = scale(my_image.read(2))
red = scale(my_image.read(3))
nir = scale(my_image.read(4))

# Stack our bands into a numpy array
rgb = np.dstack((red, green, blue)) 

# ------------------------------------------

# Check the shape
rgb.shape 

RasterioIOError: Read or write failed. ../week3/20190321_174348_0f1a_3B_AnalyticMS.tif, band 1: IReadBlock failed at X offset 22, Y offset 14: d:\bld\gdal-split_1638454034088\work\gcore\gdalrasterblock.cpp, 1125: cannot allocate 131072 bytes

Currently, the output (4213, 8341, 3) means we have an array ordered by rows, columns and then bands. 

To write this data we need the order to be bands, rows and then columns.

In [ ]:
# Let's reorder our multidimensional array.
# The numpy moveaxis function allows us to state:
# - the data.
# - the order we are labelling the data.
# - the desired array order, using the labels we just defined.
rgb_reordered = np.moveaxis(rgb, [0, 1, 2], [1, 2, 0])

# Check the shape. 
rgb_reordered.shape # Now we should have bands, rows and columns (3, 4213, 8341, ):

With the data in the correct shape, we can begin writing our data. 

In [ ]:
with rasterio.open(
    'rgb_true_color.tif',                           #our filename
    'w',                                            #write mode
    driver='GTiff',                                 #write a GeoTIFF
    height=rgb_reordered.shape[1],                  #specify the height of our image data
    width=rgb_reordered.shape[2],                   #specify the width of our image data
    count=rgb_reordered.shape[0],                   #number of bands present (e.g. )
    dtype=rgb_reordered.dtype,                      #data type
    crs=my_image.profile['crs'],                    #coordinate reference system
    transform=my_image.profile['transform']         #affine geometry transform information
    ) as my_raster_writer:
        my_raster_writer.write(rgb_reordered)       #write the data

print('Finished writing rgb_reordered')

We are also able to write a false-color version which uses near-infrared, red and green. This accentuates the areas of vegetation. 

We need to make sure we have the numpy arrays in the correct order, for bands, rows and columns:

In [ ]:
nrg = np.dstack((nir,red,green))
nrg.shape
nrg_reordered = np.moveaxis(nrg, [0, 1, 2], [1, 2, 0])

# Check the shape. 
nrg_reordered.shape # Now we should have bands, rows and columns (3, 4213, 8341, ):

In [ ]:
with rasterio.open(
    'nrg_false_color.tif',                           #our filename
    'w',                                            #write mode
    driver='GTiff',                                 #write a GeoTIFF
    height=nrg_reordered.shape[1],                  #specify the height of our image data
    width=nrg_reordered.shape[2],                   #specify the width of our image data
    count=nrg_reordered.shape[0],                   #number of bands present (e.g. )
    dtype=nrg_reordered.dtype,                      #data type
    crs=my_image.profile['crs'],                    #coordinate reference system
    transform=my_image.profile['transform']         #affine geometry transform information
    ) as my_raster_writer:
        my_raster_writer.write(nrg_reordered)       #write the data

print('Finished writing nrg_reordered')

Now go to the directory where you saved this notebook.

Open the .tif file in a piece of GIS software to view the image (e.g. ESRI Arcmap or QGIS).

Congratulations for completing this tutorial!